# Lecture 3 – Data 100, Spring 2025

Data 100, Spring 2025

[Acknowledgments Page](https://ds100.org/sp25/acks/)

A demonstration of advanced `pandas` syntax to accompany Lecture 3.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Loading the elections DataFrame
elections = pd.read_csv("data/elections.csv")

elections.head() 

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789


#### Integer-Based Extraction Using `iloc`

`iloc` selects items by row and column *integer* position.

Arguments to `.iloc` can be:
1. A list.
2. A slice (syntax is exclusive of the right hand side of the slice).
3. A single value.


In [3]:
# Select the rows at positions 1, 2, and 3.
# Select the columns at positions 0, 1, and 2.
# Remember that Python indexing begins at position 0!
elections.iloc[[1, 2, 3], [0, 1, 2]]

,Year,Candidate,Party
1,1824,John Quincy Adams,Democratic-Republican
2,1828,Andrew Jackson,Democratic
3,1828,John Quincy Adams,National Republican


In [4]:
# Index-based extraction using a list of rows and a slice of 
# column integer positions 
elections.iloc[[1, 2, 3], 0:3]

,Year,Candidate,Party
1,1824,John Quincy Adams,Democratic-Republican
2,1828,Andrew Jackson,Democratic
3,1828,John Quincy Adams,National Republican


In [5]:
# Selecting all rows using a colon
elections.iloc[:, 0:3]

,Year,Candidate,Party
0,1824,Andrew Jackson,Democratic-Republican
1,1824,John Quincy Adams,Democratic-Republican
2,1828,Andrew Jackson,Democratic
3,1828,John Quincy Adams,National Republican
4,1832,Andrew Jackson,Democratic
...,...,...,...
177,2016,Jill Stein,Green
178,2020,Joseph Biden,Democratic
179,2020,Donald Trump,Republican
180,2020,Jo Jorgensen,Libertarian


In [6]:
# Extracting data from one column
elections.iloc[[1, 2, 3], 1]

1    John Quincy Adams
2       Andrew Jackson
3    John Quincy Adams
Name: Candidate, dtype: object

In [7]:
# Extracting the value at row 0 and the second column
elections.iloc[0,1]

'Andrew Jackson'

#### Context-dependent Extraction using `[]`

We could technically do anything we want using `loc` or `iloc`. However, in practice, the `[]` operator is often used instead to yield more concise code.

`[]` is a bit trickier to understand than `loc` or `iloc`, but it achieves essentially the same functionality. The difference is that `[]` is *context-dependent*.

`[]` only takes one argument, which may be:
1. A slice of row integers.
2. A list of column labels.
3. A single column label.


If we provide a slice of row numbers, we get the numbered rows.

In [8]:
elections[3:7]

,Year,Candidate,Party,Popular vote,Result,%
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
5,1832,Henry Clay,National Republican,484205,loss,37.603628
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583


If we provide a list of column names, we get the listed columns.

In [9]:
elections[["Year", "Candidate", "Result"]]

,Year,Candidate,Result
0,1824,Andrew Jackson,loss
1,1824,John Quincy Adams,win
2,1828,Andrew Jackson,win
3,1828,John Quincy Adams,loss
4,1832,Andrew Jackson,win
...,...,...,...
177,2016,Jill Stein,loss
178,2020,Joseph Biden,win
179,2020,Donald Trump,loss
180,2020,Jo Jorgensen,loss


And if we provide a single column name we get back just that column, stored as a `Series`.

In [10]:
elections["Candidate"]

0         Andrew Jackson
1      John Quincy Adams
2         Andrew Jackson
3      John Quincy Adams
4         Andrew Jackson
             ...        
177           Jill Stein
178         Joseph Biden
179         Donald Trump
180         Jo Jorgensen
181       Howard Hawkins
Name: Candidate, Length: 182, dtype: object

### Slido Exercise

In [11]:
weird = pd.DataFrame({"a":["one fish", "two fish"], 
                      "b":["red fish", "blue fish"]})
weird

,a,b
0,one fish,red fish
1,two fish,blue fish


In [12]:
weird.iloc[1, 1]

'blue fish'

In [13]:
# weird.loc[1, 1]

In [14]:
weird.iloc[1, :]

a     two fish
b    blue fish
Name: 1, dtype: object

In [15]:
weird.loc[1, 'b']

'blue fish'

In [16]:
weird.iloc[1, 0]

'two fish'

## Dataset: California baby names

In today's lecture, we'll work with the `babynames` dataset, which contains information about the names of infants born in California.

The cell below pulls census data from a government website and then loads it into a usable form. The code shown here is outside of the scope of Data 100, but you're encouraged to dig into it if you are interested!

In [17]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "data/babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.head()

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134


## Conditional Selection

By passing in a sequence (list, array, or `Series`) of boolean values, we can extract a subset of the rows in a `DataFrame`. We will keep *only* the rows that correspond to a boolean value of `True`.

Oftentimes, we'll use boolean selection to check for entries in a `DataFrame` that meet a particular condition.

In [18]:
# First, use a logical condition to generate a boolean Series.
logical_operator = ...
logical_operator

In [19]:
# Then, use this boolean Series to filter the DataFrame.
babynames[...]

## Slido Exercises
We want to obtain the first three baby names with `count > 250`.

<img src="images/slido_1.png" width="200"/>

In [20]:
babynames.iloc[[0, 233, 484], [3, 4]]

,Name,Count
0,Mary,295
233,Mary,390
484,Mary,534


In [21]:
babynames.loc[[0, 233, 484]]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
233,CA,F,1911,Mary,390
484,CA,F,1912,Mary,534


In [22]:
babynames.loc[babynames["Count"] > 250, ["Name", "Count"]].head(3)

,Name,Count
0,Mary,295
233,Mary,390
484,Mary,534


In [23]:
babynames.loc[babynames["Count"] > 250, ["Name", "Count"]].iloc[0:2, :]

,Name,Count
0,Mary,295
233,Mary,390


## Adding, Removing, and Modifying Columns

To add a column, use `[]` to reference the desired new column, then assign it to a `Series` or array of appropriate length.

In [24]:
# Create a Series of the length of each name
babyname_lengths = babynames["Name"].str.len()

# Add a column named "name_lengths" that includes the length of each name
babynames["name_lengths"] = babyname_lengths

babynames

In [25]:
# Remove our new "Length" column
babynames.drop("name_lengths", axis="columns", inplace=True)
babynames